In [2]:
INNO = 0
def get_innovation():
    global INNO
    INNO += 1
    return INNO

ID = 0
def get_node_id():
    global ID
    ID += 1
    return ID

In [3]:
class Connect:
    def __init__(self, _in, _out, w=1, enabled=True, innov=None):
        self._in = _in
        self._out = _out
        self.weight = w
        self.enabled= enabled
        if innov is None :
            self.innovation = get_innovation()
        else :
            self.innovation = innov
    
    def __str__(self):
        s = str(self.innovation) + '  ' 
        s += str(self._in)
        s += ' -' + str(self.weight) + '-> '
        s += str(self._out)
        return s
    
    def to_node(self):
        n = Node(t='Hidden')
        c1 = Connect(self._in, n, w=self.weight, enabled=True)
        c2 = Connect(n, self._out, w=self.weight, enabled=True)
        self.enabled = False
        return (n, c1, c2)
       
    def compute(self):
        if self.enabled: self._out.sum += self.weight * self._in.funct(self._in.sum)
            
    def exists(self, i, o):
        return (self._in == i and self._out == o) or (self._in == o and self._out == i)
    
    def copy(self):
        return Connect(self._in, self._out, self.weight, self.enabled, self.innovation)
    
    
class Node:
    def __init__(self, t, id = None):
        self.sum = None
        self.type  = t
        self.funct = lambda x: 1 / (1 + math.exp(-x))
        if t == 'Input':
            self.depth = 0
        else:
            self.depth = 1
        if id is None:
            self.id = get_node_id()
        else:
            self.id = id
            
    def __str__(self):
        return str(self.id)
        
    def copy(self):
        n = Node(t=self.type)
        n.depth = self.depth
        n.id = self.id
        return n
   

In [4]:
class Brain:
    def __init__(self, _in=1, _out=1):
        self.in_out = (_in, _out)
        self.nodes = np.array([Node(t='Input') for _ in range(_in)] + [Node(t='Output') for _ in range(_out)])
        self.connects = np.array([])
        
    def __str__(self):
        s = '\nConnects:\n'
        for i in self.connects:
            s+= i.__str__() + '\n'
        s += '\nNodes:'
        for n in self.nodes:
            s += '\n' + n.type[:2] + '    ' + '  '*n.depth + str(n)
        s += '\n'
        return s
    
    def __repr__(self):
        return str(self)
    
    def plot(self):
        plt.figure(figsize=(7, 7))
        x = []
        y = []
        label = []
        for n in self.nodes:
            x.append(n.depth)
            label.append(n.id)
        y = [k for u in np.unique(x) for k in range(x.count(u))]
        for c in self.connects:
            a, b = c._in.id, c._out.id
            x0, y0 = x[label.index(a)], y[label.index(a)]
            x1, y1 = x[label.index(b)], y[label.index(b)]
            plt.arrow(x0, y0, x1-x0, y1-y0)
        plt.scatter(x, y)    
        for i, txt in enumerate(label):
            plt.annotate(txt, (np.array(x[i])+0.01, np.array(y[i])+0.01))
        plt.show()
        return self
    
    def add_random(self, nodes=0, connect=0):
        l = []
        for n in range(nodes):
            l.append(Node(t='Hidden'))
        self.nodes = np.append(self.nodes, l)
        
        l = []
        for n in range(connect):
            i = self.nodes[rd.randint(0, len(self.nodes)-1)]
            o = self.nodes[rd.randint(0, len(self.nodes)-1)]
            l.append(Connect(i, o, w=rd.random()))
        self.connects = np.append(self.connects, l)
        self.order()
        return self
        
    def full_link(self):
        c = []
        for n in self.nodes:
            if n.type[0] == 'O': # In / Hidden => Out
                for i in self.nodes:
                    if i.type[0] != 'O':
                        c.append(Connect(i, n, w=rd.random()))       
            elif n.type[0] == 'H': # In => Hidden
                for i in self.nodes:
                    if i.type[0] == 'I':
                        c.append(Connect(i, n, w=rd.random()))
        self.connects = np.array(c)
        self.order()
        return self
        
    def add_link(self, a, b, w=1, innov=None):
        if sum([c.exists(a, b) for c in self.connects]):
            return self
        i, o = None, None
        for n in self.nodes:
            if str(a) == str(n.id):
                i = n
            elif str(b) == str(n.id):
                o = n
        if i is None or o is None:
            raise Exception("Could not create  " + str(i) + " " + str(o))
        c = Connect(i, o, w, innov=innov)
        self.connects = np.append(self.connects, [c])
        self.order()
        return self
                
    def order(self):
        # reset depths
        for n in self.nodes:
            if n.type == 'Input':
                n.depth = 0
            else:
                n.depth = 1
        
        changed = True
        count = 0
        while changed:
            max_depth = max([n.depth for n in self.nodes])
            if count > 50:
                print("Looping too much:", self)
                print(len(self.nodes), "Nodes", len(self.connects), "Connections")
                break
            changed = False
            for n in self.nodes:
                for c in self.connects:
                    if c._out.id == n.id and c.enabled:
                        if c._in.depth >= n.depth:
                            n.depth = c._in.depth + 1
                            changed = True
        l = list(self.nodes)
        l.sort(key=lambda x: x.depth)
        self.nodes = np.array(l)
        return self
                        
            
    def compute(self, v, verbose=False, softmax=False):
        if len(v) != self.in_out[0]:
            raise Exception("Input size not matching")
        
        i=0
        for n in self.nodes:
            if n.type[0] == "I":
                n.sum = v[i]
                i += 1
            else:
                n.sum = 0
        
        for n in self.nodes:
            for c in self.connects:
                if c._in == n:
                    c.compute()
        o = []
        for n in self.nodes:
            if n.type[0] == "O":
                o.append(n.sum)
            if verbose: print(n.type[:2], n, n.sum)
        
        if softmax:
            o = np.exp(np.array(o))
            o = o/np.sum(o)
        return o
    
    def batch_compute(self, v, verbose=False):
        return [self.compute(i, verbose=verbose) for i in v]
            
    def mutate(self, p_add_conn=0, p_add_node=0, std=0, verbose=False):
        changed = False
        
        for c in self.connects:
            c.weight = c.weight + np.random.normal(scale=std)
        
        l = len(self.nodes)
        for i in range(l):
            if self.nodes[i].type[0] != 'O' and p_add_conn >= rd.random():
                targets = []
                for n in self.nodes:
                    exists = np.sum([c.exists(n, self.nodes[i]) for c in self.connects])
                    if n.depth >= self.nodes[i].depth and n.depth - self.nodes[i].depth < 3 and str(n.id) != str(self.nodes[i].id) and n.type[0] != 'I' and not exists:
                        targets.append(n)
                if len(targets) > 0:
                    o = targets[rd.randint(0, len(targets)-1)]
                    c = Connect(self.nodes[i], o, w=rd.random())
                    self.connects = np.append(self.connects, [c])
                    if verbose : print("Mutation: created connection", c)
                    changed=True

        l = len(self.connects)
        for i in range(l):
            if p_add_node >= rd.random():
                n, c1, c2 = self.connects[i].to_node()
                self.connects = np.append(self.connects, [c1, c2])
                self.connects = np.delete(self.connects, i)
                self.nodes = np.append(self.nodes, [n])
                changed=True
                if verbose : print("Mutation: created node", n, 'on link', c1._in.id, '->', c2._out.id)
        if changed:
            self.order()
        return self

    def reproduce(self, other):
        un = []
        s = {}
        for c in self.connects:
            s[c.innovation] = c
            un.append(c.innovation)
        o = {}
        for c in other.connects:
            o[c.innovation] = c
            un.append(c.innovation)
        un = np.unique(un)
        
        new_c = {}
        new_in = []
        new_out = []
        for n in self.nodes:
            if n.type == 'Input':
                new_in.append(n.id)
            elif n.type == 'Output':
                new_out.append(n.id)
        weights = {}
        for u in un:
            if u in s.keys() and not u in o.keys():
                new_c[u] = (s[u]._in.id, s[u]._out.id)
                weights[u] = s[u].weight
            elif not u in s.keys() and u in o.keys():
                new_c[u] = (o[u]._in.id, o[u]._out.id)
                weights[u] = o[u].weight
            elif u in s.keys() and u in o.keys():
                if np.random.randint(0, 1) == 0:
                    new_c[u] = (s[u]._in.id, s[u]._out.id)
                    weights[u] = s[u].weight
                else:
                    new_c[u] = (o[u]._in.id, o[u]._out.id)
                    weights[u] = o[u].weight
                    
        new_n = np.unique(np.array(list(new_c.values())).flatten())
        
        g = Brain()
        g.in_out = (self.in_out[0], self.in_out[1])
        new_nodes = []
        for i in new_in:
            new_nodes.append(Node(t='Input', id=i))
        for i in new_out:
            new_nodes.append(Node(t='Output', id=i))
        for i in new_n:
            if i not in new_in and i not in new_out:
                new_nodes.append(Node(t='Hidden', id=i))
        g.nodes = np.array(new_nodes)
        
        for u in new_c:
            g.add_link(new_c[u][0], new_c[u][1], weights[u], innov=u)
        
        return g
        
        
    def copy(self):
        g = Brain(0, 0)
        g.in_out = (self.in_out[0], self.in_out[1])
        g.nodes = np.array([n.copy() for n in self.nodes])
        g.connects = np.array([c.copy() for c in self.connects])
        return g